# Question Answering

In [ ]:
#Installing packages
!pip install huggingface_hub
!pip install -U transformers
!pip install datasets
!pip install datasets --upgrade
!pip install evaluate
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import transformers
import datasets
import pandas as pd
import datasets
import evaluate
import accelerate
import seaborn as sn
import numpy as np
import json
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForMaskedLM, AutoTokenizer, AutoModel, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import InputExample, InputFeatures, DataCollatorWithPadding, AutoModelForQuestionAnswering
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from transformers import AdamW
from transformers import get_scheduler
from accelerate import Accelerator
from tqdm.auto import tqdm

from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
#The two models from huggingface
distilroberta = "distilroberta-base"
climatebert = "climatebert/distilroberta-base-climate-f"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(climatebert)
model = AutoModelForQuestionAnswering.from_pretrained(climatebert)

Downloading:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

...

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.c

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/config.json
Model config RobertaConfig {
  "_name_or_path": "climatebert/distilroberta-base-climate-f",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50500
}



Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--climatebert--distilroberta-base-climate-f/snapshots/d04f8afaceb89c882a110ada795bf580627c13c4/pytorch_model.bin
Some weights of the model checkpoint at climatebert/distilroberta-base-climate-f were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were 

In [ ]:
#Import data
with open('CCMRC_train.json') as f:
    train_data = json.load(f)
with open('CCMRC_val.json') as f:
    validation_data = json.load(f)

In [ ]:
#Checking data format
#train_data#['data'][0]['paragraphs'][0]['qas'][0]['question']

In [ ]:
#Function to convert to pandas to later transform to SQuAD data structure
def qas_to_pandas(input_data, file_split):
    paragraph_text_list = []
    question_text_list = []
    answer_start_list = []
    answer_text_list = []
    qa_id_list = []
    is_impossible_list = []
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"]

            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                answer_offset = None
                is_impossible = False

                is_impossible = qa["is_impossible"]

                if file_split == "train":
                    if (len(qa["answers"]) != 1) and (not is_impossible):
                        raise ValueError(
                            "For training, each question should have exactly 1 answer."
                        )
                    if not is_impossible:
                        answer = qa["answers"][0]
                        orig_answer_text = answer["text"]
                        answer_offset = answer["answer_start"]
                    else:
                        orig_answer_text = ""
                else:
                    if not is_impossible:
                        orig_answer_text = []
                        answer_offset = []
                        for answer in qa["answers"]:
                            orig_answer_text.append(answer["text"])
                            answer_offset.append(answer["answer_start"])
                    else:
                        orig_answer_text = ""

                paragraph_text_list.append(paragraph_text)
                question_text_list.append(question_text)
                answer_start_list.append(answer_offset)
                answer_text_list.append(answer)
                qa_id_list.append(qas_id)
                is_impossible_list.append(is_impossible)

    output_df = pd.DataFrame(
        {
            "context": paragraph_text_list,
            "question": question_text_list,
            "answers": answer_text_list,
            "id": qa_id_list,
            "is_impossible": is_impossible_list,
        }
    )
    return output_df

In [ ]:
#Convert data
train_pandas = qas_to_pandas(train_data['data'],'train')
validation_pandas = qas_to_pandas(validation_data['data'],'validation')

In [ ]:
train_pandas.head()

,context,question,answers,id,is_impossible
0,"outside of the clean air act, there is support...",Is a mandate for electric production to come f...,"{'text': 'and there is ongoing interest in a ""...",665,False
1,"outside of the clean air act, there is support...",How wide ranging are climate policies across t...,"{'text': 'as of 2010, climate policies were be...",666,False
2,"outside of the clean air act, there is support...",What kind of energy standards have been implem...,"{'text': 'at latest count, 30 states have impl...",667,False
3,acknowledgments. we thank jim haywood for the ...,What was Jim Haywood thanked for?,{'text': 'we thank jim haywood for the aerosol...,1731,False
4,acknowledgments. we thank jim haywood for the ...,Where was Jonathan Gregory supported?,{'text': 'jonathan gregory was supported at th...,1732,False


In [ ]:
#Put into data dicts for huggingface models
train_final = datasets.Dataset.from_dict(train_pandas)
val_final = datasets.Dataset.from_dict(validation_pandas)

dataset_dict = datasets.DatasetDict({"train":train_final,"validation":val_final, })
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'id', 'is_impossible'],
        num_rows: 14756
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'id', 'is_impossible'],
        num_rows: 4229
    })
})

In [ ]:
#Initialize parameters
squad_v2 = True
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
#Modified code from huggingface tutorial for QA finetuning (https://huggingface.co/docs/transformers/tasks/question_answering)
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["text"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"]
            end_char = start_char + len(answers["text"])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
#Tokenize
tokenized_squad = dataset_dict.map(prepare_train_features, batched=True, remove_columns=dataset_dict["train"].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
#train model with defined hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=50,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19549
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 30550
  Number of trainable parameters = 81709826


Epoch,Training Loss,Validation Loss
1,2.665100,1.599458
2,1.579600,1.461713
3,1.407600,1.396278
4,1.311700,1.380416
5,1.144900,1.365404
6,1.082000,1.378927
7,1.020000,1.374944
8,0.977400,1.407407
9,0.924500,1.472276
10,0.825500,1.484666


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5515
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5515
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
toke

TrainOutput(global_step=30550, training_loss=0.4755104169205636, metrics={'train_runtime': 9051.4612, 'train_samples_per_second': 107.988, 'train_steps_per_second': 3.375, 'total_flos': 9.57801544697088e+16, 'train_loss': 0.4755104169205636, 'epoch': 50.0})

In [ ]:
#Save model
trainer.save_model("test-ClimateQA-trained")

Saving model checkpoint to test-ClimateQA-trained
Configuration saved in test-ClimateQA-trained/config.json
Model weights saved in test-ClimateQA-trained/pytorch_model.bin
tokenizer config file saved in test-ClimateQA-trained/tokenizer_config.json
Special tokens file saved in test-ClimateQA-trained/special_tokens_map.json


In [ ]:
#Output of training predictions
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
#Modified code from huggingface tutorial for QA finetuning (https://huggingface.co/docs/transformers/tasks/question_answering)

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=384,
        stride = 128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = dataset_dict["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset_dict["validation"].column_names
)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
#Make predictions for validation data
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5515
  Batch size = 32


In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
import collections

examples = dataset_dict["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
#Modified code from huggingface tutorial for QA finetuning (https://huggingface.co/docs/transformers/tasks/question_answering)

from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 50):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(dataset_dict["validation"], validation_features, raw_predictions.predictions)

Post-processing 4229 example predictions split into 5515 features.


  0%|          | 0/4229 [00:00<?, ?it/s]

In [ ]:
#Compute metrics for QA task
from datasets import load_metric
metric = load_metric("squad_v2" if squad_v2 else "squad")

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": [ex["answers"]]} for ex in dataset_dict["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 31.118467722865926,
 'f1': 62.34894499507892,
 'total': 4229,
 'HasAns_exact': 31.118467722865926,
 'HasAns_f1': 62.34894499507892,
 'HasAns_total': 4229,
 'best_exact': 31.118467722865926,
 'best_exact_thresh': 0.0,
 'best_f1': 62.34894499507892,
 'best_f1_thresh': 0.0}

In [ ]:
#Save results
import pickle
performance = metric.compute(predictions=formatted_predictions, references=references)

with open('results.pickle', 'wb') as handle:
    pickle.dump(performance, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
losses_epochs = trainer.state.log_history
with open('losses.pickle', 'wb') as handle:
    pickle.dump(losses_epochs, handle, protocol=pickle.HIGHEST_PROTOCOL)